# 03-202 : Question 5 Model

Train a model specifically for question 5, and then use it in conjunction with `simple_monkey` to predict the entire dataset and get a classification report.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
from typing import Tuple

import pandas as pd
import mlflow

import tensorflow_addons as tfa
import keras as k
from keras import optimizers
import keras_tuner
import keras_tuner as kt

from competition import data_preparation as dp
from competition import feature_engineering as fe
from competition import model_data as md
from competition import source_data as sd
import competition.models.simple_dense as sd_model
from competition.models.heatmap_covnet import HeatmapCovnetModel

### Configure Logging

In [3]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2023-04-01 20:24:02 INFO     Started


## Data Collection

In [4]:
# load the source training set
df_source = sd.read_csv('../data/train.csv.gz',
                        compression='gzip',
                        dtype=sd.source_dtype)

(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [5]:
# load the source training labels
df_source_labels = sd.read_csv('../data/train_labels.csv')

(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1


## Data Preparation & Cleaning

In [6]:
# prepare the main dataset
df_source = dp.prepare_main_dataset(df_source,
                                    elapsed_time_min_clip=0,
                                    elapsed_time_max_clip=3691298)

# remove sessions with problems
problem_sessions = dp.find_problem_sessions(df_source)
df_source = df_source[~df_source['session_id'].isin(problem_sessions)]

In [ ]:
# prepare the label dataset
df_source_labels = dp.prepare_label_dataset(df_source_labels)

# remove sessions with problems
df_source_labels = df_source_labels[~df_source_labels['session_id'].isin(problem_sessions)]

## Feature Engineering

Instead of duplicating the feature engineering workflow, we will use the same feature dataset created in notebook `03-123`.

In [ ]:
df_features = pd.read_pickle(
    'data/features/03-123.parquet',
    compression='gzip')

with pd.option_context('display.max_columns', None):
    display(df_features.head(3))

## Data Selection

We first combine the features with the labels as we will do data selection now based on question number as opposed to to all previous notebooks.

In [ ]:
df_combined = df_source_labels.merge(
    right=df_features, 
    on=['session_id', 'level_group'],
    how='left')

print(df_combined.shape)
with pd.option_context('display.max_columns', None):
    display(df_combined.head(6))

Create a function that will combine the datasets like we just did above and then return the dataset for the specified question.

In [ ]:
def get_question_dataset(features: pd.DataFrame,
                         labels: pd.DataFrame,
                         question_num: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns a dataset containing only the specified question_num.

    Parameters
    ----------
    features : pd.DataFrame
        The features dataset with prepared and normalized data.
    labels : pd.DataFrame
        The labels dataset containing the target variable.
    question_num : int
        The question number to filter on.

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        The filtered features and labels datasets.
    """
    # combine the features and labels datasets
    df_combined = labels.merge(
        right=features, 
        on=['session_id', 'level_group'],
        how='left')

    # filter the combined dataset on the specified question_num
    df_question = df_combined[df_combined['question_num'] == question_num]

    # convert the "heatmap" column to a list
    screen_heatmap_feature = pd.DataFrame()
    if 'screen_heatmap_feature' in df_question.columns:
        screen_heatmap_feature = df_question['screen_heatmap_feature']

    # split the combined dataset into features and labels again
    df_question_features = df_question \
        .drop(columns=['question_num', 'correct', 'screen_heatmap_feature']) \
        .drop_duplicates()
    
    # add the heatmap feature to df_question_features
    if 'screen_heatmap_feature' in df_question.columns:
        df_question_features.join(screen_heatmap_feature, how='left')

    df_question_labels = df_question[['session_id', 'question_num', 
                                     'correct', 'level_group']]        
    
    # return the filtered features and labels datasets
    return df_question_features, df_question_labels

# test the function
df_question_features, df_question_labels = get_question_dataset(features=df_features,
                                                                labels=df_source_labels,
                                                                question_num=5)

Split the data into training and test sets.

In [ ]:
random_state = 51

In [ ]:
# split the dataset into train, validation and test sets
train, val, test = md.select_sessions(
    y=df_question_labels,
    random_state=random_state,
    test_size=0.60,
    train_size=0.75)

## Model Training

### Create Datasets

In [ ]:
# set the feature list
feature_list = ['elapsed_time_sum', 'elapsed_time_max', 'elapsed_time_min', 'elapsed_time_mean', 'elapsed_time_mode']

# create the simple model dataset
features_dataset = md.get_feature_dataset(
    features=df_question_features,
    y=df_question_labels,
    feature_list=feature_list,
    train=train,
    val=val,
    test=test,
    include_question=True,
    expand_question=False)

In [ ]:
# convert the labels for multi-label classification
cat_features_dataset = md.labels_to_categorical(features_dataset)

In [ ]:
# get the shape of the question only dataset
input_data = cat_features_dataset['train']['X']
features_dataset_shape = input_data.shape[1]
print('features_dataset_shape:', features_dataset_shape)

# define the output shape
output_data = cat_features_dataset['train']['y']
output_shape = output_data.shape[1]
print('output_shape', output_shape)

### Flat Features Only

In [ ]:
# configure mlflow
mlflow.set_experiment("question-05-simple")
mlflow.keras.autolog()

In [ ]:
# create the hyperparameter object
def define_tune_parameters(hp):
    hp.Int('dense_layer_count', min_value=1, max_value=20, step=1)
    hp.Int('dense_units', min_value=640, max_value=1700, step=32)
    hp.Choice('dense_activation', values=['relu', 'tanh', 'LeakyReLU'])
    hp.Float('dense_l1_regularization', min_value=0.0, max_value=0.0005, step=0.00001)
    hp.Float('dense_l2_regularization', min_value=0.0, max_value=0.001, step=0.0001)
    hp.Float('dense_dropout', min_value=0.005, max_value=0.1, step=0.005)
    hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])

# find the best model
model = sd_model.tune_model(
    define_tune_parameters=define_tune_parameters,
    dataset=cat_features_dataset,
    max_trials=50,
    input_shape=features_dataset_shape,
    output_shape=output_shape,
    dense_layer_count='dense_layer_count',
    dense_units='dense_units',
    dense_activation='dense_activation',
    dense_l1_regularization='dense_l1_regularization',
    dense_l2_regularization='dense_l2_regularization',
    dense_dropout='dense_dropout',
    train_epochs=1000,
    train_batch_size=4000,
    train_optimizer=optimizers.Adam,
    train_learning_rate='learning_rate',
    train_loss='categorical_crossentropy',
    train_metrics=[tfa.metrics.F1Score(name='f1_score', num_classes=2, threshold=0.5, average='macro')],
    train_class_weight=None,
    tune_objective='val_f1_score',
    tune_direction='max')

model.summary()